In [ ]:
!pip3 install sqlalchemy

In [8]:
import datetime
from sqlalchemy import Column, Integer, String, Float, DateTime, create_engine
from sqlalchemy.ext.declarative import declarative_base

In [9]:
engine = create_engine('sqlite:///projektai.db')
Base = declarative_base()

In [19]:
Base.metadata.__dict__

{'tables': FacadeDict({'Projektas': Table('Projektas', MetaData(), Column('id', Integer(), table=<Projektas>, primary_key=True, nullable=False), Column('Pavadinimas', String(), table=<Projektas>), Column('Kaina', Float(), table=<Projektas>), Column('Sukurimo data', DateTime(), table=<Projektas>, default=ColumnDefault(<function datetime.utcnow at 0x117ff6830>)), schema=None)}),
 'schema': None,
 'naming_convention': immutabledict({'ix': 'ix_%(column_0_label)s'}),
 '_schemas': set(),
 '_sequences': {},
 '_fk_memos': defaultdict(list, {}),
 '_bind': None,
 'dispatch': <sqlalchemy.event.base.DDLEventsDispatch at 0x117faac40>}

In [10]:
class Projektas(Base):
    __tablename__ = "Projektas"
    id = Column(Integer, primary_key=True)
    name = Column("Pavadinimas", String)
    price = Column("Kaina", Float)
    created_date = Column("Sukurimo data", DateTime, default=datetime.datetime.utcnow)

    def __init__(self, name, price):
        self.name = name
        self.price = price

    def __repr__(self):
        return f"{self.id} {self.name} - {self.price}: {self.created_date}"


Base.metadata.create_all(engine)

### Kaip irasyti duomenis i lentele

In [11]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
# from projektas import Projektas

engine = create_engine('sqlite:///projektai.db')
Session = sessionmaker(bind=engine)
session = Session()
""":type: sqlalchemy.orm.Session"""

':type: sqlalchemy.orm.Session'

In [12]:
projektas1 = Projektas("Naujas pr.", 20000)
session.add(projektas1)
session.commit()

In [13]:
projektas2 = Projektas("2 projektas", 55000)
session.add(projektas2)
session.commit()

INSERT INTO Projektas (name, price)
VALUES ('Naujas Projektas', 20000)

## Kaip gauti duomenis is lenteles (cRud)

SELECT * FROM Projektas;

In [14]:
projektai = session.query(Projektas).all()

In [17]:
for projektas in projektai:
    print(projektas.name, projektas.price)


Naujas pr. 20000.0
2 projektas 55000.0


In [20]:
projektas1 = session.query(Projektas).get(1)


In [24]:
projektas1

1 Naujas pr. - 20000.0: 2022-11-14 16:29:25.917469

In [25]:
projektas2 = session.query(Projektas).filter_by(name="Naujas pr.").one()

Kaip ieskoti duomenu pagal sablona

In [30]:
search = session.query(Projektas).filter(Projektas.name.ilike("2%"))
search2 = session.query(Projektas).filter(Projektas.price > 1000)
search3 = session.query(Projektas).filter(Projektas.price > 1000, Projektas.name.ilike("2%"))

In [31]:
print([i for i in search])

[2 2 projektas - 55000.0: 2022-11-14 16:29:42.314332]


In [33]:
print([i for i in search2])

[1 Naujas pr. - 20000.0: 2022-11-14 16:29:25.917469, 2 2 projektas - 55000.0: 2022-11-14 16:29:42.314332]


In [34]:
print([i for i in search3])

[2 2 projektas - 55000.0: 2022-11-14 16:29:42.314332]


### Kaip pakeisti duomenis lenteleje (crUd)

In [37]:
projektas1 = session.query(Projektas).get(1)
projektas1.price = 22000
session.commit()

In [42]:
projektas2 = session.query(Projektas).filter_by(name="2 projektas").one()
projektas2.name = "2 projektas tikrai"
session.commit()

In [46]:
session.query(Projektas).filter_by(name="2 projektas tikrai").one()

2 2 projektas tikrai - 55000.0: 2022-11-14 16:29:42.314332

### Kaip istrinti duomenis lenteleje (cruD)

In [49]:
projektas1 = session.query(Projektas).filter_by(name="Naujas pr.").one()
session.delete(projektas1)
session.commit()